In [3]:
import re  # 정규 표현식 모듈 추가
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# ChromeDriver 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument("--disable-gpu")
options.add_argument("--window-size=1920,1080")
options.add_experimental_option("excludeSwitches", ["enable-logging", "enable-automation"])
options.add_argument("--disable-blink-features=AutomationControlled")

# ChromeDriver 초기화
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

# 네이버 지도 웹사이트로 이동
driver.get('https://map.naver.com/p?c=15.00,0,0,0,dh')

# 페이지 로드 대기
time.sleep(3)

# 사용자 입력 받기
search_query = input("검색할 내용을 입력하세요: ")

try:
    # 검색창 요소가 나타날 때까지 대기
    search_box = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "input.input_search"))
    )
    search_box.send_keys(search_query)
    search_box.send_keys(Keys.RETURN)

    # 결과 로드 대기
    time.sleep(5)

    # 검색 결과 iframe으로 전환
    WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it((By.ID, "searchIframe")))

    # 검색 결과 확인
    results = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.XPATH, '//*[@id="_pcmap_list_scroll_container"]/ul/li'))
    )

    if len(results) == 1:
        # 검색 결과가 하나일 때는 바로 상세 페이지로 이동
        print("검색 결과가 하나이므로 바로 상세 정보 페이지로 이동합니다.\n")
        driver.switch_to.default_content()
        WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it((By.ID, "entryIframe")))
    else:
        # 검색 결과가 여러 개일 때 첫 번째 결과 클릭
        print("검색 결과가 여러 개입니다. 첫 번째 결과를 클릭합니다.\n")
        first_result_div = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '//*[@id="_pcmap_list_scroll_container"]/ul/li[1]/div[1]'))
        )
        first_result_div.click()
        time.sleep(3)  # 클릭 후 페이지가 로드되기를 기다림

        # 메인 컨텐츠로 다시 전환
        driver.switch_to.default_content()

        # entryIframe이 로드될 때까지 대기 후 전환
        WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it((By.ID, "entryIframe")))
        time.sleep(3)

    print("첫 번째 검색 결과 클릭 완료 및 상세 정보 페이지로 이동 완료\n")

    # 가게 이름 가져오기
    store_name_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[1]"))
    )
    store_name = store_name_element.text

    # 업종 카테고리 가져오기
    category_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[2]"))
    )
    category = category_element.text

    # 별점 가져오기
    rating_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.CLASS_NAME, "place_blind"))
    )
    rating = rating_element.text

    # 방문자 리뷰 가져오기
    visitor_review_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='app-root']/div/div/div/div[2]/div[1]/div[2]/span[2]/a"))
    )
    visitor_review = visitor_review_element.text.split("리뷰")[1].strip()

    # 블로그 리뷰 가져오기
    blog_review_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='app-root']/div/div/div/div[2]/div[1]/div[2]/span[3]/a"))
    )
    blog_review = blog_review_element.text.split("리뷰")[1].strip()

    # # '리뷰' 탭 클릭하기
    # review_tab_element = WebDriverWait(driver, 15).until(
    #     EC.element_to_be_clickable((By.XPATH, "//*[@id='app-root']/div/div/div/div[4]/div/div/div/div/a[5]/span"))
    # )
    # review_tab_element.click()
    # print("리뷰 탭 클릭 완료")

    # # 몇회 몇명 참여 가져오기
    # count_element = WebDriverWait(driver, 15).until(
    #     EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[1]/div/div/div[1]/em'))
    # )
    # count = count_element.text  # 몇 회

    # participant_count_element = WebDriverWait(driver, 15).until(
    #     EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[1]/div/div/div[1]/span'))
    # )
    # participant_count = participant_count_element.text  # 몇 명 참여

    # 결과 출력
    print("가게 이름:", store_name)
    print("업종 카테고리:", category)
    print("별점:", rating)
    print("방문자 리뷰:", visitor_review)
    print("블로그 리뷰:", blog_review)
    # print("카테고리 횟수:", count)
    # print("참여 인원:", participant_count)
    print('\n')

    # # '이런 점이 좋았어요' 리스트의 첫 5개 항목과 인원수 가져오기
    # feedback_list = WebDriverWait(driver, 15).until(
    #     EC.presence_of_all_elements_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[1]/div/div/div[2]/ul/li'))
    # )
    
    # # 5개의 피드백만 가져오기
    # for i, feedback_element in enumerate(feedback_list[:5], start=1):
    #     feedback_text = feedback_element.find_element(By.XPATH, f'.//div[2]/span[1]').text
    #     print(f"피드백 {i}: {feedback_text}")

except Exception as e:
    print("오류 발생:", e)

# 필요에 따라 driver.quit()는 주석 처리된 상태로 남겨둡니다.


검색할 내용을 입력하세요:  뱅뱅막국수


검색 결과가 여러 개입니다. 첫 번째 결과를 클릭합니다.

첫 번째 검색 결과 클릭 완료 및 상세 정보 페이지로 이동 완료

가게 이름: 뱅뱅막국수
업종 카테고리: 국수
별점: 
방문자 리뷰: 3,877
블로그 리뷰: 1,953


오류 발생: Message: 



# 리뷰 전까지 가능

In [5]:
import re  # 정규 표현식 모듈 추가
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# ChromeDriver 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument("--disable-gpu")
options.add_argument("--window-size=1920,1080")
options.add_experimental_option("excludeSwitches", ["enable-logging", "enable-automation"])
options.add_argument("--disable-blink-features=AutomationControlled")

# ChromeDriver 초기화
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

# 네이버 지도 웹사이트로 이동
driver.get('https://map.naver.com/p?c=15.00,0,0,0,dh')

# 페이지 로드 대기
time.sleep(3)

# 사용자 입력 받기
search_query = input("검색할 내용을 입력하세요: ")

try:
    # 검색창 요소가 나타날 때까지 대기
    search_box = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "input.input_search"))
    )
    search_box.send_keys(search_query)
    search_box.send_keys(Keys.RETURN)

    # 결과 로드 대기
    time.sleep(5)

    # 검색 결과 iframe으로 전환
    WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it((By.ID, "searchIframe")))

    # 검색 결과 확인
    results = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.XPATH, '//*[@id="_pcmap_list_scroll_container"]/ul/li'))
    )

    if len(results) == 1:
        # 검색 결과가 하나일 때는 바로 상세 페이지로 이동
        print("검색 결과가 하나이므로 바로 상세 정보 페이지로 이동합니다.\n")
        driver.switch_to.default_content()
        WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it((By.ID, "entryIframe")))
    else:
        # 검색 결과가 여러 개일 때 첫 번째 결과 클릭
        print("검색 결과가 여러 개입니다. 첫 번째 결과를 클릭합니다.\n")
        first_result_div = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '//*[@id="_pcmap_list_scroll_container"]/ul/li[1]/div[1]'))
        )
        first_result_div.click()
        time.sleep(3)  # 클릭 후 페이지가 로드되기를 기다림

        # 메인 컨텐츠로 다시 전환
        driver.switch_to.default_content()

        # entryIframe이 로드될 때까지 대기 후 전환
        WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it((By.ID, "entryIframe")))
        time.sleep(3)

    print("첫 번째 검색 결과 클릭 완료 및 상세 정보 페이지로 이동 완료\n")

    # 가게 이름 가져오기
    store_name_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[1]"))
    )
    store_name = store_name_element.text

    # 업종 카테고리 가져오기
    category_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[2]"))
    )
    category = category_element.text

    # 별점 가져오기
    rating_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.CLASS_NAME, "place_blind"))
    )
    rating = rating_element.text

    # 방문자 리뷰 가져오기
    visitor_review_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='app-root']/div/div/div/div[2]/div[1]/div[2]/span[2]/a"))
    )
    visitor_review = visitor_review_element.text.split("리뷰")[1].strip()

    # 블로그 리뷰 가져오기
    blog_review_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='app-root']/div/div/div/div[2]/div[1]/div[2]/span[3]/a"))
    )
    blog_review = blog_review_element.text.split("리뷰")[1].strip()

    # '리뷰' 탭 클릭하기 (텍스트 기반)
    try:
        review_tab_element = WebDriverWait(driver, 15).until(
            EC.element_to_be_clickable((By.XPATH, "//span[text()='리뷰']"))
        )
        review_tab_element.click()
        print("리뷰 탭 클릭 완료")
    except Exception as e:
        print("리뷰 탭 클릭 오류:", e)

    # 몇회 몇명 참여 가져오기
    count_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[1]/div/div/div[1]/em'))
    )
    count = count_element.text  # 몇 회

    participant_count_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[1]/div/div/div[1]/span'))
    )
    participant_count = participant_count_element.text  # 몇 명 참여

    # 결과 출력
    print("가게 이름:", store_name)
    print("업종 카테고리:", category)
    print("별점:", rating)
    print("방문자 리뷰:", visitor_review)
    print("블로그 리뷰:", blog_review)
    print("카테고리 횟수:", count)
    print("참여 인원:", participant_count)
    print('\n')

    # '이런 점이 좋았어요' 리스트의 첫 5개 항목과 인원수 가져오기
    feedback_list = WebDriverWait(driver, 15).until(
        EC.presence_of_all_elements_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[1]/div/div/div[2]/ul/li'))
    )
    
    # 5개의 피드백만 가져오기
    for i, feedback_element in enumerate(feedback_list[:5], start=1):
        feedback_text = feedback_element.find_element(By.XPATH, f'.//div[2]/span[1]').text
        print(f"피드백 {i}: {feedback_text}")

except Exception as e:
    print("오류 발생:", e)

# 필요에 따라 driver.quit()는 주석 처리된 상태로 남겨둡니다.


검색할 내용을 입력하세요:  고기꾼김춘배 강남점


검색 결과가 하나이므로 바로 상세 정보 페이지로 이동합니다.

첫 번째 검색 결과 클릭 완료 및 상세 정보 페이지로 이동 완료

리뷰 탭 클릭 완료
가게 이름: 고기꾼김춘배 강남점
업종 카테고리: 돼지고기구이
별점: 
방문자 리뷰: 2,142
블로그 리뷰: 1,278
카테고리 횟수: 1,036회
참여 인원: 954명 참여


피드백 1: "음식이 맛있어요"
피드백 2: "친절해요"
피드백 3: "고기 질이 좋아요"
피드백 4: "단체모임 하기 좋아요"
피드백 5: "매장이 넓어요"
